# Sesión de Trabajo 3: LIMPIEZA DE LA BASE DE DATOS
**Asignatura:** Ciència de Dades i Intel·ligència Artificial Aplicades a la Construcció i Estructures  
**Institución:** ETSEIB - UPC  

---

## Objetivos de la sesión
Antes de usar modelos estadísticos o de aprendizaje autónomo, siempre tenemos que limpiar los datos. Ningún modelo produce resultados significativos con datos sucios. En la sesión anterior, ya hicimos el primer paso para realizar la limpieza: detectar aquellas variables que requieren de correcciones. La sesión de trabajo actual se centra en corregir/rellenar/eliminar aquellos datos incorrectos/innecesarios/inconsistentes/nulos identificados perdiendo la mínima información posible. La sesión se estructura en diversas tareas.

* **S3.T1.** Tratamiento de los datos no relevantes

* **S3.T2.** Tratamiento de los datos inconsistentes

* **S3.T3.** Tratamiento de las observaciones nulas (missing data)

* **S3.T4.** Tratamiento de las observaciones atípicas (outliers)

No existe un orden ni estrategia establecida para encarar esta tarea. En general, se intenta empezar por aquellas acciones más simples y de mayor impacto y poco a poco se va hacia las acciones más específicas.
El objetivo es claro: obtener una BBDD relevante, consistente y sin observaciones nulas ni atípicas que sea lo más completa posible. Hay muchos caminos alternativos para alcanzarlo.

---


### S3.T0. Importar la base de datos
* Conectamos con Google Drive.
* Cargamos el archivo `BBDD_ST2.csv`.
* Optimizamos los tipos de datos (`float32` e `int32`) para no saturar la RAM de Colab.

⏳ *Tiempo de ejecución estimado: 1-2 minutos (dependiendo de la velocidad de conexión a Drive).*


In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# Montar Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Ruta y Carga
ruta_drive = '/content/drive/MyDrive/Sostenibilidad/BBDD_ST2.csv'
df = pd.read_csv(ruta_drive, low_memory=False)

# Optimización
float64_cols = df.select_dtypes(include='float64').columns.tolist()
df[float64_cols] = df[float64_cols].astype('float32')
int64_cols = df.select_dtypes(include='int64').columns.tolist()
df[int64_cols] = df[int64_cols].astype('int32')

print("Base de datos cargada y optimizada.")


---

### S3.T1. Tratamiento de datos NO RELEVANTES

* **S3.T1.1.** Identificar y Eliminar columnas no informativas debido a su alta repetitividad de sus observaciones.
* **S3.T1.2.** Identificar y Eliminar columnas no informativas debido a su nula relación con el fenómeno de estudio.
* **S3.T1.3.** Eliminar observaciones duplicadas. Esto se debería hacer al final de la limpieza de la BBDD.

---

* **S3.T1.1.** Identificar y Eliminar columnas no informativas debido a su alta repetitividad de sus observaciones.

    

>>EJEMPLO: Columnas no informativas por repetición de valores o NaNs

In [ ]:
df['SOLAR TERMICA'].value_counts(dropna=False) * 100 / len(df.index)

>>EJEMPLO: Variables con pocos datos pero valiosos. Antes de eliminar una variable, mejor analizar su distribución con atención y asegurar que no aporta información relevante para el fenómeno de estudio

In [ ]:
# 1. Preparación de datos
ind_ve = df[df['VEHICLE ELECTRIC'] == 'SI'].index
data_ve = df.loc[ind_ve, "Qualificació de consum d'energia primaria no renovable"].value_counts(normalize=True, dropna=False).sort_index()
data_global = df["Qualificació de consum d'energia primaria no renovable"].value_counts(normalize=True, dropna=False).sort_index()

# 2. Definir una paleta de colores fija para que las letras (A, B, C...) coincidan siempre
# Usamos un mapa de colores 'RdYlGn' (Rojo-Amarillo-Verde) invertido para eficiencia energética
colores = plt.cm.RdYlGn_r(np.linspace(0, 1, len(data_global)))

# 3. Crear la figura con 1 fila y 2 columnas
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico 1: Subgrupo con Vehículo Eléctrico
data_ve.plot(
    kind="pie",
    ax=ax1,
    autopct='%1.1f%%',
    startangle=90,
    colors=colores,
    wedgeprops={'edgecolor': 'white'}
)
ax1.set_title("Edificios con VEHÍCULO ELÉCTRICO\n(Submuestreo)", fontweight='bold')
ax1.set_ylabel("") # Elimina el texto lateral innecesario

# Gráfico 2: Distribución Global
data_global.plot(
    kind="pie",
    ax=ax2,
    autopct='%1.1f%%',
    startangle=90,
    colors=colores,
    wedgeprops={'edgecolor': 'white'}
)
ax2.set_title("Distribución GLOBAL de la BBDD\n(Población Total)", fontweight='bold')
ax2.set_ylabel("")

>>*TAREA: Completa la definición de la variable "cols_baja_varianza" con las variables (columnas) que tienen falta de variabilidad (Repetitividad o NaNs).*

>>*Criterio: Columnas con >95% de valores idénticos o nulos.*

In [ ]:
cols_baja_varianza = [
    'SOLAR TERMICA',              # 100% "NO"
    'INFORME_INS_TECNICA_EDIFICI', # 99.83 % NaN
    # ... Añade aquí el resto de variables, poniendo al lado el motivo de su inclusión ...
]

>>Eliminar columnas no informativas por repetición de valores o NaNs

In [ ]:
df = df.drop(columns=cols_baja_varianza, errors='ignore')

* **S3.T1.2.** Identificar y Eliminar columnas no informativas debido a su nula relación con el fenómeno de estudio.

In [ ]:
# Lista de variables a eliminar por nula relación
cols_no_relacionadas = [
    'NUM_CAS',              # ID administrativo de la Generalitat
    'REFERENCIA CADASTRAL', # No influye en la física del edificio
    # --- Añade aquí el resto de variables detectadas ---
]

# Ejecución de la eliminación
df = df.drop(columns=cols_no_relacionadas, errors='ignore')

* **S3.T1.3.** Eliminar observaciones duplicadas. Esto se debería repetir al final de la limpieza de la BBDD.

In [ ]:
df = df.drop_duplicates()  # Eliminar observaciones duplicadas

---

### S3.T2. Tratamiento de datos INCONSISTENTES

* **S3.T2.1.** Uniformizar uso de las mayúsculas/minúsculas y corregir errores de escritura en variables categóricas.
* **S3.T2.2.** Errores de formato en direcciones, números, ...

---


* **S3.T2.1.** Uniformizar uso de las mayúsculas/minúsculas y corregir errores de escritura en variables categóricas.

>Las variables categóricas a menudo presentan "suciedad" debido a la entrada manual: mezcla de mayúsculas, errores ortográficos o sinónimos. Para limpiar estas variables, seguiremos estas cuatro estrategias:

>1. **Normalización de texto:** Convertimos todo a minúsculas para que el sistema no trate como categorías distintas palabras que solo varían en su grafía (ej: "Vivienda" vs "vivienda").

>>> `df['COL'] = df['COL'].str.lower()`

>2. **Mapeo de sinónimos (Agrupación):** Unificamos términos que significan lo mismo o agrupamos categorías muy similares mediante un diccionario de corrección.

>>> `df['COL'] = df['COL'].replace({'antiguo': 'nuevo'})`

>3. **Filtrado de errores:** Si detectamos valores imposibles, símbolos extraños o errores que no pueden corregirse por falta de información, eliminamos esas filas (observaciones).

>>> `df = df.drop(df[df['COL'] == 'error'].index, axis=0)`

>4. **Tratamiento del ruido:** En variables con muchísima dispersión (como municipios o tipos de uso muy específicos), aparecen categorías con poquísimas muestras. Corregirlas manualmente requiere un esfuerzo excesivo para el beneficio que aportan, y estadísticamente pueden "confundir" al modelo de IA al no tener suficientes ejemplos para aprender un patrón fiable. Estrategia: Establecemos un umbral mínimo (ej. 400 observaciones). Si una categoría no llega a ese mínimo, eliminamos esas filas para quedarnos con un dataset más robusto y limpio.

>>>`counts = df['US_EDIFICI'].value_counts()`

>>>`to_remove = counts[counts <= 400].index`

>>>`df = df[~df['US_EDIFICI'].isin(to_remove)]`

>5. **Evaluar siempre la relación esfuerzo/beneficio:** A veces, una variable tiene tantos errores ortográficos o categorías caóticas (ej. POBLACIO) que el tiempo necesario para limpiarla no compensa la información que aporta. En estos casos, es más eficiente eliminar la columna completa.

>>>`print(df['POBLACIO'].value_counts())`

In [ ]:
# EJEMPLO DE TRATAMIENTO COMPLETO DE UNA VARIABLE: Limpieza de NOM_PROVINCIA

print("ESTADO INICIAL DE 'NOM_PROVINCIA':")
print(df['NOM_PROVINCIA'].value_counts(dropna=False))

# 1. Uniformizar a minúsculas
df['NOM_PROVINCIA'] = df['NOM_PROVINCIA'].str.lower()

# 2. Eliminar observaciones erróneas (ejemplo: valor '0')
# Identificamos índices y eliminamos esas filas
ind_erroneos = df[df['NOM_PROVINCIA'] == '0'].index
df = df.drop(ind_erroneos, axis=0)

# 3. Reubicar columna al principio para facilitar la inspección visual
col_name = "NOM_PROVINCIA"
df.insert(0, col_name, df.pop(col_name))

print("\nESTADO FINAL TRAS LIMPIEZA:")
print(df['NOM_PROVINCIA'].value_counts(dropna=False))

In [ ]:
# EJEMPLO DE TRATAMIENTO COMPLETO DE UNA VARIABLE: Limpieza de US_EDIFICI

print("ESTADO INICIAL DE 'US_EDIFICI':")
print(df['US_EDIFICI'].value_counts(dropna=False))

df['US_EDIFICI'].value_counts(dropna=False)  # Distribución de valores de US_EDIFICI
df['US_EDIFICI'] = df['US_EDIFICI'].str.lower()  # Convertir a minúsculas
# 2. Diccionario de correcciones (Agrupación de categorías)
correcciones_uso = {
    'terciario': 'terciari',
    'terciari: locals, oficines...': 'terciari',
    'vivienda unifamiliar': 'habitatge unifamiliar',
    'habitatge unifamiliar aïllat': 'habitatge unifamiliar',
    'habitatge unifamiliar adossat': 'habitatge unifamiliar',
    'bloque de viviendas': "bloc d'habitatges plurifamiliar",
    'bloque de viviendas plurifamiliar': "bloc d'habitatges plurifamiliar",
    "bloc d'habitatges": "bloc d'habitatges plurifamiliar",
    "habitatge plurifamiliar": "bloc d'habitatges plurifamiliar",
    "vivienda individual en bloque de viviendas": "habitatge individual en bloc d'habitatges",
    "bloque de viviendas": "bloc d'habitatges plurifamiliar"
}
# Aplicamos el mapeo
df['US_EDIFICI'] = df['US_EDIFICI'].replace(correcciones_uso)

# 3. Eliminación de ruido (Categorías con <= 400 observaciones)
counts = df['US_EDIFICI'].value_counts()
to_remove = counts[counts <= 400].index
df = df[~df['US_EDIFICI'].isin(to_remove)]

# 4. Reubicar columna al principio (es una buena práctica)
df.insert(0, 'US_EDIFICI', df.pop('US_EDIFICI'))

print("\nESTADO FINAL TRAS LIMPIEZA:")
print(df['US_EDIFICI'].value_counts(dropna=False))


>TAREA: Repite el proceso en una celda de código independiente para cada variable categórica que consideres que se debe limpiar (crea tantas celdas de código como necesites).

In [ ]:
#Insertar código aquí para la limpieza de la variable 1

In [ ]:
#Insertar código aquí para la limpieza de la variable 2

In [ ]:
#Insertar código aquí para la limpieza de la variable 3 ...

* **S3.T2.2.** Errores de formato en direcciones, números, ...

>A menudo, los datos numéricos vienen "contaminados" por formatos regionales (comas en lugar de puntos) o errores de exportación que convierten números en texto. En esta sección, aprenderemos a estandarizar estos formatos.

>#### Estrategias comunes:
>1. **Ceros a la izquierda:** Los códigos postales o IDs deben tener una longitud fija (ej. `08001`). Si se cargan como números, el `0` inicial desaparece.
>2. **Corrección de separadores:** En Python, los decimales usan el punto (`.`). Si el dataset usa comas (`,`) para miles o decimales incorrectos, hay que eliminarlas o sustituirlas.
>3. **Casteo de tipos (Casting):** Forzar a que una variable sea `float` (número con decimales) o `str` (texto) para que las funciones de análisis no den error.

*En esta tarea os damos todo el código ya realizado. No tenéis que realizar ni implementar código nuevo, simplemente ejecutar el que os proporcionamos.*

In [ ]:
# --- Limpieza de CODI_POSTAL ---

print("Muestra de Códigos Postales iniciales:")
print(df['CODI_POSTAL'].head())

# El objetivo es que todos tengan 5 dígitos y sean texto.
# 1. Eliminar valores nulos (NaN) ya que no podemos inventar un código postal
df = df.dropna(subset=['CODI_POSTAL'])

# 2. Asegurar formato: De número -> a Entero (quita decimal .0) -> a Texto -> Rellenar ceros
df['CODI_POSTAL'] = df['CODI_POSTAL'].astype(int).astype(str).str.zfill(5)

# 3. Mover al principio
df.insert(0, 'CODI_POSTAL', df.pop('CODI_POSTAL'))

print("Muestra de Códigos Postales limpios:")
print(df['CODI_POSTAL'].head())

In [ ]:
# --- Limpieza de Emissions de CO2 y Energia primària no renovable ---

print("Verificación de tipos de datos inicial:")
print(df[["Emissions de CO2", "Energia primària no renovable"]].dtypes)

# Eliminamos la coma para que Python pueda tratarlos como números decimales (float).

# Tratamiento de Emissions de CO2
df["Emissions de CO2"] = [float(str(i).replace(",", "")) for i in df["Emissions de CO2"]]

# Tratamiento de Energia primària no renovable
df["Energia primària no renovable"] = [float(str(i).replace(",", "")) for i in df["Energia primària no renovable"]]

print("Verificación de tipos de datos final:")
print(df[["Emissions de CO2", "Energia primària no renovable"]].dtypes)

# Visualizamos las primeras filas para confirmar que son números limpios
display(df[["Emissions de CO2", "Energia primària no renovable"]].head())

In [ ]:
# --- Limpieza de Valor finestres y aillaments CTE ---

df['VALOR FINESTRES CTE'] = df['VALOR FINESTRES CTE'].astype(str)  # Convertir el tipo de datos de VALOR FINESTRES CTE a str
df['VALOR AILLAMENTS CTE'] = df['VALOR AILLAMENTS CTE'].astype(str)  # Convertir el tipo de datos de VALOR AILLAMENTS CTE a str

---

### S3.T3. Tratamiento de observaciones nulas (missing data)

No todos los nulos se deben tratar igual. Dependiendo de cuántos falten y qué tipo de variable sea, elegiremos una de estas 5 alternativas:

1. **Borrado por densidad:** Si a una fila le faltan casi todos los datos y no tenemos criterio para rellenar los valores, no es útil. Borrar observaciones (filas) es "barato".
2. **Borrado puntual:** Si faltan muy pocos datos en una columna crítica (ej. `ZONA CLIMATICA`), borramos esas filas, nunca la columna entera.
3. **Imputación por tendencia central:** Sustituir nulos por la **mediana** (numéricas) o la **moda** (categóricas).
4. **Categoría "Missing":** Crear una categoría nueva llamada "DESCONOCIDO" para no perder la información de las otras columnas.
5. **Borrado de columna:** Si la columna aporta poco y está casi vacía.

---



>Diagnòstico 1: Ver nº NaNs por columnas

In [ ]:
# 1. Recuento de NaNs por columna

# Configuramos pandas para que no ponga límites al número de filas mostradas al imprimir
pd.set_option('display.max_rows', 100) # Suficiente para tus 69 columnas

print("--- Recuento total de NaNs por columna (69 columnas) ---")
# Calculamos los nulos y los mostramos todos
nulos_totales = df.isnull().sum().sort_values(ascending=False)
print(nulos_totales)

# Restablecemos la configuración por defecto (opcional, para no saturar futuras celdas)
pd.reset_option('display.max_rows')

>Diagnóstico 2: Ver nº NaNs por filas

In [ ]:
# Creamos una serie con el conteo de nulos por cada fila
nans_por_fila = df.isnull().sum(axis=1)

# Contamos cuántas filas hay con 0 nulos, con 1 nulo, con 2, etc.
distribucion_nans = nans_por_fila.value_counts().sort_index()

# Visualización
distribucion_nans.plot.bar(
    figsize=(10, 5),
    title="Diagnóstico 2: ¿Cuántos campos vacíos tienen los edificios?",
    xlabel="Número de NaNs detectados en una fila",
    ylabel="Cantidad de edificios (registros)",
    color='skyblue',
    edgecolor='black'
)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

>> Alternativa 1: Borrar filas con exceso de NaNs. Por ejemplo, si un edificio tiene más de 6 campos vacíos, lo descartamos

In [ ]:
umbral = 6
df = df[df.isnull().sum(axis=1) <= umbral]

>>Alternativa 2: Borrar filas puntuales

In [ ]:
df.drop([5], axis=0)  # Eliminar observación con í­ndice 5

>>Alternativa 3: Borrar filas que tengan un NaN en una determinada columna. Apropiado si hay muy pocos NaNs en la columna.

In [ ]:
df = df.dropna(subset=['ZONA CLIMATICA'])

>>Alternativa 4: Borrar columnas poco informativas que contengan observaciones nulas

In [ ]:
df = df.drop(df.columns[3], axis=1)  # Eliminar columna 3

>>Alternativa 5: Asignar un valor a la observación nula. Distintos casos según variables continuas o categóricas

In [ ]:
med = df['METRES_CADASTRE'].median()  # Calcular la mediana de la variable METRES_CADASTRE
df['METRES_CADASTRE'] = df['METRES_CADASTRE'].fillna(med)  # Sustituimos NaNs por la mediana

In [ ]:
top = df['SISTEMA BIOMASSA'].describe()['top']  # Calcular el valor más frecuente de la variable SISTEMA BIOMASSA
df['SISTEMA BIOMASSA'] = df['SISTEMA BIOMASSA'].fillna(top)  # Sustituimos NaNs por el campo mÃ¡s comÃºn

In [ ]:
df['XARXA DISTRICTE'] = df['XARXA DISTRICTE'].fillna('MISSING')  # Sustituimos NaNs una categorí­a nueva

>TAREA: Trata las observaciones NaNs de tu base de datos con adoptando la alternativa más adiente en cada caso. Justifica tu decisión con un comentario al lado del código. El objetivo es terminar sin ningún NaN en tu BBDD.


EJEMPLO Limpieza de ZONA CLIMATICA.

In [ ]:
# EJEMPLO DE TRATAMIENTO COMPLETO DE UNA VARIABLE: Zona climática
# 1. Diagnóstico: ¿Cuántos nulos tenemos realmente?
num_missing = df["ZONA CLIMATICA"].isnull().sum()
print(f"Valores nulos iniciales en ZONA CLIMATICA: {num_missing}")

# 2. Decisión y Ejecución:
# Como solo hay 46 nulos sobre miles de registros, la pérdida es despreciable (<1%).
# Optamos por eliminar las filas (Alternativa 3).
df = df[df['ZONA CLIMATICA'].notna()]

# 3. Organización:
# Movemos la columna al principio para facilitar la inspección visual
col_name = "ZONA CLIMATICA"
first_col = df.pop(col_name)
df.insert(0, col_name, first_col)

num_missing = df["ZONA CLIMATICA"].isnull().sum()
print(f"Valores nulos finales en ZONA CLIMATICA: {num_missing}")

In [ ]:
# Introduce aquí el código para la limpieza de otra variable (utiliza una celda de código por variable)

---

### S3.T4. Tratamiento de Outliers en Variables Continuas

No todos los nulos se deben tratar igual. Dependiendo de cuántos falten y qué tipo de variable sea, elegiremos una de estas 5 alternativas:

Un **outlier** en una variable continua es un valor que se aleja drásticamente del comportamiento general. En nuestro contexto (edificios), suele indicar:
* **Errores de entrada:** Un edificio de 100,000 $m^2$ que en realidad tenía 100.
* **Casos atípicos:** Edificios singulares que no representan la norma y que pueden sesgar las predicciones de la IA.

En ambos casos, hay que eliminar estas observaciones de nuestra base de datos.

---



Paso 1: Diagnóstico Visual y Estadístico

In [ ]:
# 1. Diagnóstico estadístico y visual
print(df['METRES_CADASTRE'].describe())

# Visualizamos la densidad para detectar "colas" largas en el gráfico
sns.kdeplot(data=df['METRES_CADASTRE'])
plt.title("Distribución de Metros Cuadrados")
plt.show()

Paso 2: Control de filas a eliminar

In [ ]:
# 1. Definir límites basados en el diagnóstico anterior
limite_inferior = 5
limite_superior = 50000

# 2. Identificar y Contar
ind_out = df[(df['METRES_CADASTRE'] < limite_inferior) | (df['METRES_CADASTRE'] > limite_superior)].index

print(f"Registros que se eliminarán: {len(ind_out)}")
print(f"Porcentaje de pérdida: {(len(ind_out) / len(df)) * 100:.2f}%")

Paso 3: Eliminación y validación

In [ ]:
# 3. EJECUCIÓN (Solo si el porcentaje es razonable)
if len(ind_out) > 0:
    df = df.drop(ind_out, axis=0)
    # Reubicamos la columna para ver el resultado
    df.insert(0, 'METRES_CADASTRE', df.pop('METRES_CADASTRE'))
    print(f"\n Eliminación completada. Nuevo tamaño: {df.shape[0]} filas.")
else:
    print("\n No se han eliminado registros.")

TAREA: Elimina los outliers de tu base de datos

In [ ]:
# Introduce aquí el código para la limpieza de otra variable (utiliza una celda de código por variable)

Eliminar observaciones duplicadas.

In [ ]:
n_antes_dupl = len(df)
df = df.drop_duplicates()
print(f"Registros duplicados eliminados: {n_antes_dupl - len(df)}")

Guardar BBDD limpia

In [ ]:
# Usamos la misma carpeta donde cargamos la BBDD_ST2
ruta_salida = '/content/drive/MyDrive/Sostenibilidad/BBDD_ST3.csv'
df.to_csv(ruta_salida, index=False)

# 🏁 TAREAS FINALES: Wrap-up de la Sesión de Trabajo 3

**Descripción:** Una vez finalizado el proceso de depuración, es necesario auditar la calidad de la base de datos resultante. Contesta a las siguientes preguntas comparando el estado inicial (**BBDD_ST2**) y el estado final tras la limpieza (**BBDD_ST3**).


### 1. Auditoría de Dimensiones
Realiza un balance del volumen de datos tras aplicar los filtros de nulos, duplicados y outliers.

* **Número de filas:**
    * *Antes (ST2):* `______`
    * *Después (ST3):* `______`
* **Número de columnas:**
    * *Antes (ST2):* `______`
    * *Después (ST3):* `______`

---

### 2. Definición del Modelo del proceso de certificación
Identifica las variables que serán el motor de tu modelo en la próxima sesión.

* **Variables INPUTS (Entradas):** Escribe el nombre de las columnas que usarás como posibles INPUTS (Entradas) en el proceso de certificación.
    * *Respuesta:*
* **Variables OUTPUTS (Salidas):** Escribe el nombre de las columnas que usarás como posibles OUTPUTS (Salidas) del proceso de certificación.
    * *Respuesta:*

---

### 3. Impacto de la limpieza por Variable

A continuación, detalla los cambios específicos en todas las variables no eliminadas.

#### **A. Variables Categóricas (Unificación y Ruido)**
| Variable | Nº Categorías Inicial (ST2) | Nº Categorías Final (ST3) | Comentario (ej: ¿Qué agrupaste?) |
| :--- | :---: | :---: | :--- |
| `US_EDIFICI` | | | |
| `NOM_PROVINCIA` | | | |
| `ZONA CLIMATICA` | | | |

#### **B. Variables Continuas (Eliminación de Outliers)**
| Variable | Valor Mín / Máx (ST2) | Valor Mín / Máx (ST3) | ¿Por qué elegiste esos límites? |
| :--- | :---: | :---: | :--- |
| `METRES_CADASTRE` | | | |
| `compacitat` | | | |
| `Emissions de CO2` | | | |
| `Energia primària...`| | | |